In [29]:
import spotipy
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from dotenv import load_dotenv
import re
from tqdm import tqdm

load_dotenv('../secrets/.env', override=True)

%run useful_functions.ipynb
%run spotify_scraping.ipynb

spotify = spotipy.Spotify(client_credentials_manager=SpotifyClientCredentials())

In [33]:
broadcast_playlists = get_broadcast_playlists()
search = get_spotify_search()
tracks = get_spotify_tracks()
albums = get_spotify_albums()
artists = get_spotify_artists()

tracks_to_run = broadcast_playlists[['Track Name', 'Artist Name']].sort_values(['Artist Name','Track Name'], ascending =  True).drop_duplicates(subset=['Artist Name','Track Name'], keep='first')
tracks_to_run = tracks_to_run.merge(search[['Original Search Artist','Original Search Track']], how='left', left_on=['Artist Name','Track Name'], right_on=['Original Search Artist','Original Search Track'], copy=False,indicator=True)
tracks_to_run = tracks_to_run[tracks_to_run['_merge']!="both"][['Artist Name','Track Name']]
tracks_to_run

,Artist Name,Track Name
0,,Faking Jazz Together
1,+HIRS+,Y'all Are Precious.
2,...From The Benthic Zone,Edwin
8,128kps,D4y n N1t3 (Phasmid's Unnecessary Mix)
15,24Hour Experience,Together
...,...,...
12673,Бони НЕМ,And I Love Her
12674,አለማየሁ እሸቴ,Alteleyeshegnem
12675,“Little” Louie Vega,All My Love
12682,上々颱風,Let It Be


In [34]:
max_iter = 10000

for i, (index, row) in enumerate(tracks_to_run[0:max_iter].iterrows()):
    print("Iteration ",i+1," of ",len(tracks_to_run.index),":",row['Artist Name'],":", row['Track Name'])
    try:
        (temp_tracks,temp_albums,temp_artists,temp_search) = get_track_info(row['Artist Name'],row['Track Name'])
    except:
        (temp_tracks,temp_albums,temp_artists,temp_search) = (pd.DataFrame(),pd.DataFrame(),pd.DataFrame(),pd.DataFrame())
    tracks = pd.concat([tracks,temp_tracks])
    albums = pd.concat([albums,temp_albums])
    artists = pd.concat([artists,temp_artists])
    search = pd.concat([search,temp_search])
    
del(i,
    max_iter,
    index,
    row,
    temp_albums,
    temp_artists,
    temp_tracks,
    temp_search,
    tracks_to_run)  

Iteration  1  of  1053 :  : Faking Jazz Together
Error with:   : Faking Jazz Together
Iteration  2  of  1053 : +HIRS+ : Y'all Are Precious.
Error with:  +HIRS+ : Yall Are Precious.
Iteration  3  of  1053 : ...From The Benthic Zone : Edwin
Error with:  ...From The Benthic Zone : Edwin
Iteration  4  of  1053 : 128kps : D4y n N1t3 (Phasmid's Unnecessary Mix)
Error with:  128kps : D4y n N1t3
Iteration  5  of  1053 : 24Hour Experience : Together
Error with:  24Hour Experience : Together
Iteration  6  of  1053 : 2DB : Mesmerised
Error with:  2DB : Mesmerised
Iteration  7  of  1053 : 801 : Tomorrow Never Knows
Error with:  801 : Tomorrow Never Knows
Iteration  8  of  1053 : 9TRANE : Made It Thru (9TRANE'S Back to '91 Mix)
Error with:  9TRANE : Made It Thru
Iteration  9  of  1053 : A.A. Williams : Mumurs
Error with:  A.A. Williams : Mumurs
Iteration  10  of  1053 : A/P : Terrace Fever (Dance System's World Cup '02 Mix)
Error with:  A/P : Terrace Fever


In [35]:
search = search.drop_duplicates(subset=['Original Search Artist', 'Original Search Track'], keep='last')
albums = albums.drop_duplicates(subset='Album ID', keep='last')
tracks = tracks.drop_duplicates(subset='Track ID', keep='last')
artists = artists.drop_duplicates(subset='Artist ID', keep='last')

tracks.to_pickle("../data/spotify_tracks.pkl")
albums.to_pickle("../data/spotify_albums.pkl")
artists.to_pickle("../data/spotify_artists.pkl")
search.to_pickle("../data/spotify_search.pkl")

In [40]:
temp = search.merge(tracks, how = 'left', on = 'Track ID')[['Track Name', 'Original Search Track', 'Original Search Artist']].applymap(lambda s: s.lower() if type(s) == str else s)
temp = temp[temp['Track Name'] != temp['Original Search Track']]
temp.to_excel('../data/temp.xlsx')
del(temp)
#CAN IMPROVE UPON MATCHING